In [5]:
import os

while 'resco_benchmark' not in os.listdir():
    os.chdir('..')

from resco_benchmark.multi_signal import MultiSignal
from resco_benchmark.config.agent_config import agent_configs
from resco_benchmark.config.map_config import map_configs
from resco_benchmark.config.mdp_config import mdp_configs

args = agent_configs['IDQN']
map_name = 'ingolstadt1'
map_config = map_configs[map_name]

env = MultiSignal(run_name = 'test', map_name = map_name, 
                  net = os.path.join('resco_benchmark/', map_config['net']), state_fn = args['state'], 
                  reward_fn = args['reward'], route=None, gui=False, 
                  end_time=3600, step_length=10, yellow_length=4, step_ratio=1, 
                  max_distance=200, lights=(), log_dir='/', libsumo=False, warmup=0, gymma=False)

ingolstadt1 resco_benchmark/environments/ingolstadt1/ingolstadt1.sumocfg drq_norm wait_norm
 Retrying in 1 seconds
lights 1 ('gneJ207',)
Step #57600.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 0 ACT 0 BUF 0)                   


OSError: [Errno 30] Read-only file system: '/test-ingolstadt1-0-drq_norm-wait_norm'

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# REMARK: no activation function in GraphConvLayer!!!
class GraphConvLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x, adj_matrix):
        """
        x: [B, NUM_NODES, INPUT_DIM]
        adj_matrix: [NUM_NODES, NUM_NODES]

        output: [B, NUM_NODES, OUTPUT_DIM]
        """

        batch_size, num_nodes, _ = x.size()

        adj_matrix_hat = adj_matrix + torch.eye(n=num_nodes)
        deg_mat_inv_square_root = torch.diag(torch.sum(adj_matrix, dim=-1)** (-0.5))
        adj_matrix_modified = deg_mat_inv_square_root @ adj_matrix_hat @ deg_mat_inv_square_root
        
        output = self.linear(adj_matrix_modified @ x)
        
        return output

class GraphNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GraphNeuralNetwork, self).__init__()
        assert num_layers >= 2, "Number of layers needs to be greater or equal to 2"
        self.num_middle_layers = num_layers-2
        self.first_layer = GraphConvLayer(input_dim, hidden_dim)
        self.middle_layers = nn.ModuleList([GraphConvLayer(hidden_dim, hidden_dim) for _ in range(self.num_middle_layers)])
        self.last_layer = GraphConvLayer(hidden_dim, output_dim)

    def forward(self, x, adj_matrix):
        """
        x: [B, NUM_NODES, INPUT_FEATURES]
        adj_matrix: [NUM_NODES, NUM_NODES]

        output: [B, NUM_NODES, OUTPUT_DIM]
        """
        x = F.relu(self.first_layer(x, adj_matrix))
        for i in range(self.num_middle_layers):
            x = F.relu(self.middle_layers[i](x, adj_matrix))
        output = self.last_layer(x, adj_matrix)
        return output

# Correctness test
input_dim = 16
hidden_dim = 32
output_dim = 2
batch_size = 10
num_nodes = 5
num_layers = 6

x = torch.randn(batch_size, num_nodes, input_dim)
adj_matrix = torch.randint(0, 2, (num_nodes, num_nodes)).float()

gnn = GraphNeuralNetwork(input_dim, hidden_dim, output_dim, num_layers)

output = gnn(x, adj_matrix)
print(output.shape)


torch.Size([10, 5, 2])


In [7]:
"""
Plan:

1. Get obs from env.reset()

2. Check shape of observation i.e. number of crossings, num of features for each crossing

3. Create adjacency matrix.

3. Set params for GNN so that a single observation can be passed through and yield actions
for each of the crossings. Observation may be something like (NUM_CROSSINGS, CROSSING_DIM_1, CROSSING_DIM_2, ...)

4. The output format of GNN needs to be checked and aligned with the env requirements. Might be of 
the shape (NUM_CROSSINGS, ACTION_SPACE) <- Q-values for pair (state, action_1), (state, action_2)...

Rough notes:
- should be consistent with SharedAgent
"""


'\nPlan:\n\n1. Get obs from env.reset()\n\n2. Check shape of observation i.e. number of crossings, num of features for each crossing\n\n3. Create adjacency matrix.\n\n3. Set params for GNN so that a single observation can be passed through and yield actions\nfor each of the crossings. Observation may be something like (NUM_CROSSINGS, CROSSING_DIM_1, CROSSING_DIM_2, ...)\n\n4. The output format of GNN needs to be checked and aligned with the env requirements. Might be of \nthe shape (NUM_CROSSINGS, ACTION_SPACE) <- Q-values for pair (state, action_1), (state, action_2)...\n\n5. \n'